In [ ]:
import sys, os, h5py, time, itertools
sys.path.append('../')
import numpy as np
from scipy.optimize import differential_evolution
from iminuit import Minuit
from problem import Problem

# Set-Up

In [ ]:
# Load the problem class and set it up with the signal model that contains the candidate line.
perseus = Problem('./Data/ReducedData/Perseus_Data.h5',
                  './Data/ModelComponents/APEC_Grid.npz',
                  './Data/ModelComponents/Absorption_Grid.npy',
                  './Fitting/SigDict.npz',
                  min_energy =3, max_energy = 6,)

# Optimization Methods

`chi2(A)` evaluates the $\chi^2$ for the model for the specified signal flux `A` in units of $10^{-6}$ photons/cm$^2$/s. If desired it will also return the best-fit (at fixed `A`) parameter vector.

In [ ]:
# Getting the index of the singal line. We need this later.
sig_index = np.where(problem.bounds[0] == -100)[0][0]

# Returns chi^2
def chi2(A, return_fit = False):
    
    profile_bounds = np.copy(problem.bounds)
    profile_bounds[:, ,sig_index] = A # Fix signal parameter to `A`
    
    out = differential_evolution(loss, profile_bounds.T, atol = 0, tol = 1e-4,
                                 disp = True, popsize=50)
    m = Minuit.from_array_func(loss, out.x, limit = free_bounds.T, errordef = 1)
    m.migrad()
    
    if return_fit:
        return m.fval, m.np_values()
    
    return m.fval

# Calculate the profiled likelihood

In [ ]:
# profiling range
profile_range = np.linspace(-30, 60, 128)
chi2_vals = np.zeros_like(profile_range)

for i, A in enumerate(profile_range):
    chi2_vals = chi2(A)
    print(i, A)


You have now calculated a profiled likelihood and should save the results.

In [ ]:
np.save(np.vstack((profile_range, chi2_vals)))